# Calculating IRR

**Formula**

$$NPV = \sum_{t=0}^{n} \frac{C_t}{(1 + IRR)^t} = 0$$

Where we set NPV to 0, use FCFE (Free Cash Flow from Equity) as Cash from at time *t*. 

FCFE will be adjusted by debt levels of each company -> if stable, use FCFF as a proxy, if not, adjust with change in total debt

**IRR Function**

In [93]:
import numpy as np
import numpy_financial as npf
import pandas as pd

def compute_irr(cash_flows):
    """
    cash_flows: list or array, t=0..T
      cash_flows[0] = -price_today (negative)
      cash_flows[1:] = annual FCFE (or FCFF if used as proxy), last may include terminal value
    
    Returns:
        IRR as decimal (e.g. 0.14 == 14%).
    """
    irr = npf.irr(cash_flows)
    if irr is None:
        raise ValueError("IRR solver failed. Check cash flows (signs, zeros).")
    return irr


**Controls**

In [94]:
# -----------------------------
# 0) HIGH-LEVEL CONTROLS
# -----------------------------

# Choose one of: "nike", "lulu", "atz_cn"
STOCK = "atz_cn"

# Horizon: how many *explicit* forecast years you want to use for IRR
# Example: 2 years ahead (2026, 2027) + terminal value at 2027
HORIZON_YEARS = 5

print("Stock:", STOCK)
print("Horizon (years):", HORIZON_YEARS)


Stock: atz_cn
Horizon (years): 5


**Conensus and Debt Levels**

In [95]:
# -----------------------------
# 1) RAW INPUTS FROM BLOOMBERG 
# All values in *absolute* units (e.g. CAD or USD)
# -----------------------------

# Example structure: {year: value}
# FCFF from Bloomberg "Free Cash Flow" (consensus) for future years (Consensus)
# FCFF from Bloomberg "Free Cash Flow" consensus
# In absolute currency units (multiply 'millions' by 1e6)

fcff_dict = {
    "nike": {
        2026: 2_635.2 * 1_000_000,  # FY 2026E FCF (USD mm)
        2027: 3_765.5 * 1_000_000,  # FY 2027E
        2028: 4_572.1 * 1_000_000,  # FY 2028E
        2029: 4_965.6 * 1_000_000,  # FY 2029E
        2030: 5_502.7 * 1_000_000,  # FY 2030E
    },
    "lulu": {
        2026: 1_152.6 * 1_000_000,  # FY 2026E FCF (USD mm)
        2027: 1_245.6 * 1_000_000,  # FY 2027E
        2028: 1_405.3 * 1_000_000,  # FY 2028E
        2029: 1_400.1 * 1_000_000,  # FY 2029E
        2030: 1_503.7 * 1_000_000,  # FY 2030E
    },
    "atz_cn": {
        2026:   384.9 * 1_000_000,  # FY 2026E FCF (CAD mm)
        2027:   601.5 * 1_000_000,  # FY 2027E
        2028:   679.1 * 1_000_000,  # FY 2028E
        2029:   811.1 * 1_000_000,  # FY 2029E
        2030:   956.7 * 1_000_000,  # FY 2030E
    },
}


# Total Debt dictionary (can be partial; ΔDebt assumed 0 if missing)
# Include last *known* prior year so we can compute ΔDebt to first forecast year.
debt_dict = {
    "nike": {
        # Example: debt basically stable, but we include it for completeness.
        2025: 23_362.0 * 1_000_000,
        2026: 23_362.0 * 1_000_000,   
        2027: 23_362.0 * 1_000_000,
    },
    "lulu": {
        2025: 3279.2 * 1_000_000,
        2026: 3279.2  * 1_000_000,
        2027: 3279.2  * 1_000_000,
    },
    "atz_cn": {
        2025: 1_361.2 * 1_000_000,
        2026: 1_361.2 * 1_000_000,
        2027: 1_361.2 * 1_000_000,
    }
}

wacc_dict = {
    "nike": 9.5043,
    "lulu": 9.7772,
    "atz_cn": 13.6206,

}

# Current equity value (price * shares) as of today, per name
# These must be set from current market data.
equity_value_dict = {
    "nike":  88_692_081_760.00,   
    "lulu":  24_681_182_826.00,   
    "atz_cn": 13_432_629_145.00,   
}

print("Market Cap (equity value)")
for tic in equity_value_dict:
    print(f"{tic}: {equity_value_dict[tic]}")


Market Cap (equity value)
nike: 88692081760.0
lulu: 24681182826.0
atz_cn: 13432629145.0


**Build FCFF and Debt Series**

In [96]:
# -----------------------------
# 2) BUILD PER-STOCK SERIES
# -----------------------------

# Years in sorted order, limited to HORIZON_YEARS
all_years = sorted(fcff_dict[STOCK].keys())
years = all_years[:HORIZON_YEARS]

fcff = np.array([fcff_dict[STOCK][y] for y in years], dtype=float)

# Debt can be missing. If so, we will treat ΔDebt as 0.
debt_years_raw = debt_dict.get(STOCK, {})
debt_years_sorted = sorted(debt_years_raw.keys())
debt = {y: debt_years_raw[y] for y in debt_years_sorted}

print("Years:", years)
print("FCFF:", fcff)
print("Debt entries:", debt)


Years: [2026, 2027, 2028, 2029, 2030]
FCFF: [3.849e+08 6.015e+08 6.791e+08 8.111e+08 9.567e+08]
Debt entries: {2025: 1361200000.0, 2026: 1361200000.0, 2027: 1361200000.0}


**Compute Change in Debt and FCFE per year (under assumptions)**

In [97]:
# -----------------------------
# 3) COMPUTE ΔDebt AND FCFE
# -----------------------------

delta_debt = []
fcfe = []

for y in years:
    # Try to compute ΔDebt = Debt_y - Debt_(y-1)
    if (y in debt) and ((y - 1) in debt):
        d_delta = debt[y] - debt[y - 1]
    else:
        d_delta = 0.0  # assume stable leverage if we can't see both years
    
    delta_debt.append(d_delta)

# Now construct FCFE, depending on stock:
# - Nike: treat FCFF ≈ FCFE (ignore ΔDebt)
# - LULU: FCFF base, FCFE as sensitivity (we'll compute both)
# - ATZ: use FCFE = FCFF + ΔDebt (important)

fcff = np.array(fcff, dtype=float)
delta_debt = np.array(delta_debt, dtype=float)

if STOCK == "nike":
    fcfe = fcff.copy()  # FCFF ≈ FCFE
elif STOCK == "lulu":
    fcfe = fcff + delta_debt  # sensitivity case; base IRR will still be FCFF
else:  # "atz_cn"
    fcfe = fcff + delta_debt  # leverage materially rising

print("ΔDebt per year:", delta_debt)
print("FCFF per year:", fcff)
print("FCFE per year:", fcfe)

ΔDebt per year: [0. 0. 0. 0. 0.]
FCFF per year: [3.849e+08 6.015e+08 6.791e+08 8.111e+08 9.567e+08]
FCFE per year: [3.849e+08 6.015e+08 6.791e+08 8.111e+08 9.567e+08]


**Terminal Value**

In [98]:
# -----------------------------
# 4) TERMINAL VALUE ASSUMPTION
# -----------------------------

# Simple perp-growth on last FCFE. You may replace these with your actual DCF terminal assumptions.
terminal_growth = 0.02   # 2% long-run growth
terminal_discount_guess = 0.09  # 10% just for TV construction; IRR will be solved independently

last_fcfe = fcfe[-1]
tv = last_fcfe * 1.05 / wacc_dict[STOCK] - terminal_growth

print("Last FCFE:", last_fcfe)
print("Terminal value (TV):", tv)


Last FCFE: 956700000.0
Terminal value (TV): 73751156.31672525


**Build Cash Flow List**

In [99]:
# -----------------------------
# 5) BUILD CASH FLOW STREAMS FOR IRR
# -----------------------------

equity_value_today = equity_value_dict[STOCK]  # market cap (or price * shares)

# FCFF-based IRR stream (for Nike/LULU base case)
cash_flows_fcff = [-equity_value_today] + list(fcff[:-1]) + [fcff[-1] + tv]

# FCFE-based IRR stream
cash_flows_fcfe = [-equity_value_today] + list(fcfe[:-1]) + [fcfe[-1] + tv]

print("Cash flows (FCFF-based):")
print(cash_flows_fcff)
print("\nCash flows (FCFE-based):")
print(cash_flows_fcfe)


Cash flows (FCFF-based):
[-13432629145.0, np.float64(384900000.0), np.float64(601500000.0), np.float64(679100000.0), np.float64(811100000.0), np.float64(1030451156.3167253)]

Cash flows (FCFE-based):
[-13432629145.0, np.float64(384900000.0), np.float64(601500000.0), np.float64(679100000.0), np.float64(811100000.0), np.float64(1030451156.3167253)]


**Compute IRR**

In [100]:
# -----------------------------
# 6) COMPUTE IRR
# -----------------------------

irr_fcff = compute_irr(cash_flows_fcff)
irr_fcfe = compute_irr(cash_flows_fcfe)

print(f"\nIRR (FCFF-based) for {STOCK}: {irr_fcff:.4%}")
print(f"IRR (FCFE-based) for {STOCK}: {irr_fcfe:.4%}")

# Which IRR do we use as expected_return_annual?
# - Nike: IRR_FCFF ≈ IRR_FCFE, use FCFF-based (cleaner)
# - LULU: use FCFF-based as primary; FCFE-based as sensitivity
# - ATZ: use FCFE-based (leverage matters)

if STOCK == "nike":
    expected_return_annual = irr_fcff
elif STOCK == "lulu":
    expected_return_annual = irr_fcff  # base
else:  # "atz_cn"
    expected_return_annual = irr_fcfe  # leverage-adjusted

print(f"\nChosen expected_return_annual for {STOCK}: {expected_return_annual:.4%}")



IRR (FCFF-based) for atz_cn: -30.2171%
IRR (FCFE-based) for atz_cn: -30.2171%

Chosen expected_return_annual for atz_cn: -30.2171%
